# Aloha Online Bulk Upload Guide
*This guide is for setting up the program on your machine, maintenance, or modification purposes.*

## Prerequisites:
- [Python](https://www.python.org/downloads/) (3.3 or higher)
- [Pip](https://jupyter.org/install) installer for downloading integral packages
    - Pandas library
    - Jupyter notebook, if updating this guide
    
## The notebook shows how to:
- Import the necessary libraries
- Stage the data that the program is run on
- Run the program
- Call API to upload site modifications

### Optional: install jupyter notebook (classic)

In [ ]:
pip install notebook

*from the command line or terminal, navigate to the desired folder location using "cd file\path\to\your\folder" (reverse slashes if using terminal) and then typing "jupyter notebook" to create a new notebook*

### Install the pandas library

In [ ]:
pip install pandas

Use the "xlrd" package to read Excel files, install using:

In [ ]:
pip install xlrd

Use the "json" package for formatting the web call, install using:

In [ ]:
pip install json

Import the libraries into the program

In [21]:
import pandas as pd, xlrd, json
from itertools import count

Read the Excel file that contains the site hours

In [22]:
excel_file = r'HoursChanges_BulkUpload.xlsx' # set the path to the upload template (when in the right folder with template, shift-click and copy the filepath)
hours_sheet = 0 # set the sheet number with the hours on it, left to right counting from 0
AO_Site_ID = 0 # set the column number with the AO Site IDs in it, counting from 0
import_items = pd.read_excel(excel_file, sheet_name=hours_sheet, index_col=AO_Site_ID, usecols="C:AS")
df = pd.DataFrame(data=import_items)

*Note: This presumes the Excel file is in the same directory (a.k.a. folder) as the program, but you can put the full filepath in the method call to ensure the program finds it*

*Note: If you recieve a strange error (this does not include 'File not found') and no output, ensure the file is saved as an 'Excel Workbook' and not a 'Strict XML Open Spreadsheet' by opening the file, Saving As, and using the drop-down menu to select Excel Workbook.*

This creates the JSON call container and a list containing weekdays (to loop through):

In [23]:
sites = { "Sites": [] } # this creates the container for all the sites
days = [ "SUNDAY", "MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY"] # simple list for changing the day of the week dynamically

The main for lopp iterates through the template, gathering and organizing store data into a format that JSON can interpret:

In [24]:
for i, site in enumerate(import_items):
    if i < len(import_items.index) and i < 3: # remove 'and' when not testing
        sites['Sites'].append( { "SiteID": int(import_items.index.values[i]), "StoreHours": [], "DeliveryHours": [] }) # creates a single site and puts it in Sites container
        for j, day in zip(count(step=6), days): # loops thru the site's store hours for each day of week and adds it to a single site
            sites['Sites'][i]['StoreHours'].append( {"DayOfWeek": day, "IsClosed": str(df.iloc[i,j+2]), "OpeningTime": str(df.iloc[i,j]), "ClosingTime": str(df.iloc[i,j+1]) } ) # site hours
            sites['Sites'][i]['DeliveryHours'].append( {"DayOfWeek": day, "IsClosed": str(df.iloc[i,j+5]), "Delivery1Start": str(df.iloc[i,j+3]), "Delivery1End": str(df.iloc[i,j+4]) } ) # delivery hours

Finally, the JSON call is put into a test file in the same directory for ease of use:

In [25]:
with open("JSON-call.txt", "w") as file: # creates & writes the JSON call to a text file
    file.write(json.dumps(sites, indent=4))

Note: This will overwrite a file with the same name or previously run version of the .txt file

### And your done! The text file should now contain your API call